# Basic Prompt Structures Tutorial

## Overview

This tutorial explores two essential prompt structures used in AI interactions:
1. Single-turn prompts
2. Multi-turn prompts (conversations)

We'll utilize Google's Gemini via OpenRouter and LangChain to illustrate these concepts.

## Motivation

Grasping various prompt structures is essential for effective AI communication. Single-turn prompts excel in quick, direct queries, while multi-turn prompts facilitate more nuanced, context-rich exchanges. Proficiency in these structures enhances the versatility and efficacy of AI applications across diverse scenarios.

## Key Components

1. **Single-turn Prompts**: One-time interactions with the AI model.
2. **Multi-turn Prompts**: Sequential exchanges that preserve context.
3. **Prompt Templates**: Standardized structures for consistent AI querying.
4. **Conversation Chains**: Techniques for maintaining context across multiple interactions.

## Setup

First, let's import the necessary libraries and set up our environment.

In [1]:
from os import getenv
from typing import List

from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI

load_dotenv()


# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="google/gemini-flash-1.5",
)

## 1. Single-turn Prompts

Single-turn prompts are one-shot interactions with the language model. They consist of a single input (prompt) and generate a single output (response).

In [2]:
single_turn_prompt = "What are the three primary colors?"
print(llm.invoke(single_turn_prompt).content)

The three primary colors in additive color mixing (like on a computer screen) are **red, green, and blue**.



Now, let's use a PromptTemplate to create a more structured single-turn prompt:

In [3]:
structured_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Provide a brief explanation of {topic} and list its three main components.",
)

chain = structured_prompt | llm
print(chain.invoke({"topic": "color theory"}).content)

Color theory is a set of guiding principles that explains how colors mix, match, and create different effects on the human eye.  It's used in various fields like art, design, and even marketing to create visually appealing and impactful results.  It's based on the understanding of how colors relate to each other, both physically (wavelengths of light) and psychologically (emotional responses).

The three main components of color theory are:

1. **Hue:** This refers to the pure color, like red, blue, or green.  It's the basic name we give to a color.

2. **Saturation:** This describes the intensity or purity of a color. A highly saturated color is vibrant and strong, while a less saturated color appears duller or more grayed.

3. **Brightness (or Value):** This refers to the lightness or darkness of a color.  A high-brightness color is light, almost white, while a low-brightness color is dark, almost black.



You can learn more about LangChain Expression Language from [LCEL](https://python.langchain.com/docs/concepts/lcel/)

## 2. Multi-turn Prompts (Conversations)

Multi-turn prompts involve a series of interactions with the language model, allowing for more complex and context-aware conversations.

In [4]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """In memory implementation of chat message history."""

    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


# Here we use a global variable to store the chat message history.
# This will make it easier to inspect it to see the underlying results.
store = {}


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | llm

conversation = RunnableWithMessageHistory(
    chain,
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

print(
    conversation.invoke(
        input={
            "question": "Hi, I'm learning about space. Can you tell me about planets?"
        },
        config={"configurable": {"session_id": "foo"}},
    ).content
)
print(
    conversation.invoke(
        input={"question": "What's the largest planet in our solar system?"},
        config={"configurable": {"session_id": "foo"}},
    ).content
)
print(
    conversation.invoke(
        input={"question": "How does its size compare to Earth?"},
        config={"configurable": {"session_id": "foo"}},
    ).content
)

Planets are fascinating celestial bodies!  To understand them, let's break it down:

**What is a Planet?**

The International Astronomical Union (IAU) defines a planet as a celestial body that:

1. **Orbits the Sun:**  It travels around the Sun in a roughly elliptical path.
2. **Has sufficient mass to assume hydrostatic equilibrium:** This means it's massive enough for its own gravity to pull it into a nearly round shape.
3. **Has cleared the neighborhood around its orbit:**  This means it's gravitationally dominant in its orbital zone; it's either pulled in or flung out any other objects of comparable size.  This is the key difference between a planet and a dwarf planet.

**Types of Planets in Our Solar System:**

Our solar system has eight planets, categorized into two main types:

* **Inner, rocky planets (terrestrial planets):** These are smaller, denser planets closer to the Sun, composed mostly of rock and metal. They include:
    * **Mercury:** The smallest and closest to the Su

Let's compare how single-turn and multi-turn prompts handle a series of related questions:

In [6]:
# Single-turn prompts
prompts = [
    "What is the capital of France?",
    "What is its population?",
    "What is the city's most famous landmark?",
]

print("Single-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    print(f"A: {llm.invoke(prompt).content}\n")

# Multi-turn prompts
print("Multi-turn responses:")
for prompt in prompts:
    print(f"Q: {prompt}")
    response = conversation.invoke(
        input={"question": prompt}, config={"configurable": {"session_id": "bar"}}
    ).content
    print(f"A: {response}\n")

Single-turn responses:
Q: What is the capital of France?
A: Paris


Q: What is its population?
A: Please specify what "it" refers to.  I need to know the name of the city, country, region, or other entity you're asking about to tell you its population.


Q: What is the city's most famous landmark?
A: Please specify which city you're asking about.


Multi-turn responses:
Q: What is the capital of France?
A: Paris


Q: What is its population?
A: The population of Paris is a bit tricky to define precisely because it depends on what area you're considering.

* **Paris proper (intra-muros):**  Around 2.1 million people.  This refers to the city's administrative boundaries.

* **Greater Paris (Île-de-France region):**  Over 12 million people. This includes the suburbs and surrounding areas.

So, the answer depends on the level of detail you need.


Q: What is the city's most famous landmark?
A: The Eiffel Tower


